# Dataset Preprocessing

### In this process, we convert the annotations from the VisDrone dataset format to the YOLO annotation format.

VisDrone Annotation Format
The VisDrone dataset provides annotations in a specific format, where each object in an image is represented by a line with the following fields:


- [x_min, y_min, width, height, class_id, difficult, truncated, occluded]

    - **x_min, y_min**: The coordinates of the top-left corner of the bounding box.
    - **width, height**: The width and height of the bounding box.
    - **class_id**: The numerical identifier for the object class.
    - **difficult**: A flag indicating if the object is difficult to detect.
    - **truncated**: A flag indicating if the object is truncated (partially visible).
    - **occluded**: A flag indicating if the object is occluded (partially blocked by another object).

YOLO Annotation Format
In contrast, the YOLO annotation format represents bounding boxes relative to the image size, with normalized values between 0 and 1. The format for each object is:

- [class_id, x_center, y_center, width, height]

    - **class_id**: The numerical identifier for the object class.
    - **x_center, y_center**: The normalized coordinates of the bounding box center.
    - **width, height**: The normalized width and height of the bounding box.

In [1]:
import os
from pathlib import Path
from PIL import Image
from tqdm import tqdm
from ultralytics.utils.downloads import download

def visdrone2yolo(directory):
    def convert_box(size, box):
        # Convert VisDrone box to YOLO xywh box
        dw = 1. / size[0]
        dh = 1. / size[1]
        return (box[0] + box[2] / 2) * dw, (box[1] + box[3] / 2) * dh, box[2] * dw, box[3] * dh

    (directory / 'labels').mkdir(parents=True, exist_ok=True)  # create labels directory
    pbar = tqdm((directory / 'annotations').glob('*.txt'), desc=f'Converting {directory}')
    for f in pbar:
        img_size = Image.open((directory / 'images' / f.name).with_suffix('.jpg')).size
        lines = []
        with open(f, 'r') as file:  # read annotation.txt
            for row in [x.split(',') for x in file.read().strip().splitlines()]:
                if row[4] == '0':  # VisDrone 'ignored regions' class 0
                    continue
                cls = int(row[5]) - 1
                box = convert_box(img_size, tuple(map(int, row[:4])))
                lines.append(f"{cls} {' '.join(f'{x:.6f}' for x in box)}\n")
                with open(str(f).replace(f'{os.sep}annotations{os.sep}', f'{os.sep}labels{os.sep}'), 'w') as fl:
                    fl.writelines(lines)  # write label.txt

# Define the directory containing your annotations
directory = Path("C:/Users/juank/AI Projects/VisDrone/dataset")  # root dataset dir

# Convert VisDrone annotations to YOLO labels for the following datasets
for d in ['VisDrone2019-DET-train', 'VisDrone2019-DET-val', 'VisDrone2019-DET-test-dev']:
    visdrone2yolo(directory / d)  # convert VisDrone annotations to YOLO labels

Converting C:\Users\juank\AI Projects\VisDrone\dataset\VisDrone2019-DET-train: 6471it [01:23, 77.21it/s] 
Converting C:\Users\juank\AI Projects\VisDrone\dataset\VisDrone2019-DET-val: 548it [00:07, 71.03it/s] 
Converting C:\Users\juank\AI Projects\VisDrone\dataset\VisDrone2019-DET-test-dev: 1610it [00:18, 86.01it/s] 


# Note:

After converting the annotations from the VISDRONE format in the "annotations" folder to the YOLO format in the "labels" folder, the "annotations" folder is deleted to prevent any issues during model training.